In [2]:
!ls ../feat_selection/ | grep json

select_optuna_catb.json
select_optuna_lgbm_v1.json
select_optuna_lgbm_v2.json
select_optuna_lgbm_v3.json


In [3]:
import json
import numpy as np
import pandas as pd

***
### feature selection overlap

In [4]:
# Load the 4 feature selection JSON files
with open('../feat_selection/select_optuna_lgbm_v1.json', 'r') as f:
    fsv1 = json.load(f)
    
with open('../feat_selection/select_optuna_lgbm_v2.json', 'r') as f:
    fsv2 = json.load(f)
    
with open('../feat_selection/select_optuna_catb.json', 'r') as f:
    fsv3     = json.load(f)
    
with open('../feat_selection/select_optuna_lgbm_v3.json', 'r') as f:
    fsv4 = json.load(f)

In [10]:
fsv1_numerical, fsv1_categorical = fsv1["numerical"], fsv1["categorical"]
fsv2_numerical, fsv2_categorical = fsv2["numerical"], fsv2["categorical"]
fsv3_numerical, fsv3_categorical = fsv3["numerical"], fsv3["categorical"]
fsv4_numerical, fsv4_categorical = fsv4["numerical"], fsv4["categorical"]

# Create a list of all feature selection versions
fs_versions = [
    ('fsv1', fsv1_numerical, fsv1_categorical),
    ('fsv2', fsv2_numerical, fsv2_categorical), 
    ('fsv3', fsv3_numerical, fsv3_categorical),
    ('fsv4', fsv4_numerical, fsv4_categorical)
]

# Iterate through each version
for i, (name, num, cat) in enumerate(fs_versions):
    print('-'*100)
    print(f"{name}:")
    print(f"Number of numerical features: {len(num)}")
    print(f"Number of categorical features: {len(cat)}")
    print("\nCategorical features:")
    print(cat)
    
    # Compare with other versions
    for j, (other_name, other_num, other_cat) in enumerate(fs_versions):
        if i != j:  # Don't compare with self
            # Calculate overlap counts and percentages
            num_overlap_set = set(num) & set(other_num)
            cat_overlap_set = set(cat) & set(other_cat)
            num_overlap_pct = len(num_overlap_set) / len(num) * 100
            cat_overlap_pct = len(cat_overlap_set) / len(cat) * 100
            
            print(f"\nMatch with {other_name}:")
            print(f"Numerical features overlap: {len(num_overlap_set)} features ({num_overlap_pct:.1f}%)")
            print(f"Categorical features overlap: {len(cat_overlap_set)} features ({cat_overlap_pct:.1f}%)")


----------------------------------------------------------------------------------------------------
fsv1:
Number of numerical features: 281
Number of categorical features: 6

Categorical features:
['agent1', 'agent2', 'agent1_playout', 'agent1_score_bounds', 'agent2_exploration_const', 'agent2_score_bounds']

Match with fsv2:
Numerical features overlap: 144 features (51.2%)
Categorical features overlap: 3 features (50.0%)

Match with fsv3:
Numerical features overlap: 142 features (50.5%)
Categorical features overlap: 4 features (66.7%)

Match with fsv4:
Numerical features overlap: 145 features (51.6%)
Categorical features overlap: 4 features (66.7%)
----------------------------------------------------------------------------------------------------
fsv2:
Number of numerical features: 296
Number of categorical features: 5

Categorical features:
['agent1', 'agent2', 'agent1_selection', 'agent1_playout', 'agent2_selection']

Match with fsv1:
Numerical features overlap: 144 features (48.6

***
### importance selection

***